In [1]:
import cv2
import numpy as np
from PIL import Image

def makeCutouts(mkvPath):
    # Get filename, without extension
    filename = mkvPath.split('\\')[-1].split('.')[0]

    print(filename)
    # Step 1
    cap = cv2.VideoCapture(mkvPath)

    # Set the frame number to skip. If you want every frame, set this to 1
    frame_skip = 10

    frame_count = 0

    # define the RGB tuple for our color to be replaced
    target_color = (0, 0, 0)  # black
    tolerance = 50  # define your tolerance level

    def color_within_tolerance(color_a, color_b, tolerance):
        return sum((a - b) ** 2 for a, b in zip(color_a, color_b)) <= tolerance ** 2

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_skip != 0:
            continue

        # Step 3
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA))

        # Step 4
        datas = image.getdata()

        new_data = []
        for item in datas:
            if color_within_tolerance(item[:3], target_color, tolerance):
                new_data.append((item[0], item[1], item[2], 0))  # Make transparent
            else:
                new_data.append(item)
        image.putdata(new_data)

        # Step 5
        bbox = image.getbbox()
        if bbox:
            image = image.crop(bbox)

        # Step 6
        image.save(f'cutouts/{filename}/{filename}_{frame_count//frame_skip}.png')

    cap.release()


In [ ]:
# For every .mkv in model_recordings, make a cutout
import os
for file in os.listdir('model_recordings'):
    if file.endswith('.mkv'):
        makeCutouts(os.path.join('model_recordings', file))

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os
directory = 'model_recordings'
video_files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
with ProcessPoolExecutor() as executor:
        print(video_files)
        executor.map(makeCutouts, video_files)

In [ ]:
# Or, in parallel
from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    for file in os.listdir('model_recordings'):
        if file.endswith('.mkv'):
            executor.submit(makeCutouts, os.path.join('model_recordings', file))


In [ ]:
for file in os.listdir('model_recordings'):
    # make a directory in cutouts for every .mkv
    if file.endswith('.mkv'):
        filename = file.split('.')[0]
        os.mkdir(f'cutouts/{filename}')